In [2]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
processed_df = pd.read_csv("EBITDA_new.csv")
processed_df.head(3)

,ticker,fiscalQuarter,commonSharesOutstanding_,commonSharesTraded_,capitalExpenditure_,netIncome_,operatingIncome_,shareholdersEquity_,totalAssets_,EPS_,...,EBITDA,salePrice,realGDPSA,m2SA,m2m1GrowthGap,m2Velocity,primeRate,EBITDA_lag1,year,quarter
0,ACAN,2013Q1,-0.000933,12.247579,0.0,-0.073272,-0.095742,-0.203545,0.0,0.0,...,50.369,13.564960,-0.867729,9.261987,0.172473,1.581,3.25,NaN,2013,1
1,ACAN,2013Q2,-0.000933,12.247579,0.0,-0.073210,-0.095754,-0.203545,0.0,0.0,...,42.617,13.651304,-0.849252,9.272702,-0.640550,1.572,3.25,50.369,2013,2
2,ACAN,2013Q3,-0.000933,12.247579,0.0,-0.073519,-0.095692,-0.203545,0.0,0.0,...,46.147,13.650265,-0.790315,9.286502,-0.328302,1.571,3.25,42.617,2013,3


In [7]:
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

# Select features and target variable
# features exclude:
features = [col for col in processed_df.columns if col not in [ 'EBITDA', 'ticker', 'companyName','fiscalQuarter']]
target = 'EBITDA'

# Remove missing values
processed_df = processed_df.dropna(subset=features + [target])

# Split the dataset based on year

train_df = processed_df[processed_df['year'] <= 2020]
test_df = processed_df[processed_df['year'] > 2020]

# Update train and test sets
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]
# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'verbose': -1
}

# Train the model
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, test_data], num_boost_round=100)

# Make predictions
y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

RMSE: 447.603598634377
R2 Score: 0.9263088331192654
